In [ ]:
# !find ./log/* -type f -not -name "stepNum*" -exec rm -f {} \;
# clean the json format log and it's corresponding csv file

In [ ]:
import pandas
import matplotlib.pyplot as plt
import json

In [ ]:
class Log():
    def __init__(self, json_log_path):
        self.json_log_path = json_log_path
        self.param = json.load(open(self.json_log_path, 'r'))
        self.param_names = list(self.param.keys())
    def __getitem__(self, key):
        assert key in self.param.keys(), "key not in param"
        return self.param[key]
    
    def get_data(self,nrows=None):
        if nrows is None:
            return pandas.read_csv(self.param["data"])
        else:
            return pandas.read_csv(self.param["data"], nrows=nrows)
    
    def __str__(self) -> str:
        return f"Log({self.json_log_path})"

    def serialize(self) -> str:
        """
        序列化的时候忽略属性"other"
        """
        temp_json = self.param.copy()
        temp_json.pop("other")
        temp_json.pop("data")
        return json.dumps(temp_json)
 

# log1 = Log("./log/20231221223430_1c31d0eb-0f82-4be8-824d-2af22816be25.json")
# print(log1.param_names, log1["normId"])
# log1["data"]
# log1.get_data()

In [ ]:
import pandas
import os

NROWS = 100000
logs = list()
for root, dirs, files in os.walk("./log/"):
    for file in files:
        if file.endswith(".json"):
            logs.append(Log(os.path.join(root, file))) 
            print(os.path.join(root, file))
print(logs)

# 对logs按照norm的大小排序
logs = sorted(logs, key=lambda log:log["normId"])

# 限制读取行数为 100000 行
data = logs[0].get_data(NROWS)
data

In [ ]:
end = data.shape[0]



# sort logs by normId
logs = sorted(logs, key=lambda log:log["normId"])
print("logs_num:", len(logs))
filtered_logs = list()
for log in logs:
    if log["stepNum"] == 200:
        filtered_logs.append(log)
print("filtered_logs_num:", len(filtered_logs))

    # 遍历 data_paths
for log in filtered_logs:
    # 读取数据
    data = log.get_data(NROWS)
    # 第一列为横轴，第二列为纵轴画曲线，label为列名
    # 设置cmap
    colors = ['red', 'blue', 'green', 'yellow', 'black', 'purple', 'pink', 'orange', 'gray', 'brown', 'cyan', 'magenta',
              'olive', 'lime', 'teal', 'navy']
    plt.figure(figsize=(20, 6))
    for i in range(1, 17):
        plt.plot(data.iloc[:end, 0], data.iloc[:end, i], label=data.columns.values[i], color=colors[i - 1])
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.30), ncol=4, fancybox=True, shadow=True)
    plt.title("normId: " + str(log["normId"]))
    # plt.savefig('./norm10-c-ur.png', dpi=600)

In [ ]:
for log in logs:
    # 读取数据
    data = log.get_data(NROWS)
    # 第一列为横轴，第二列为纵轴画曲线，label为列名
    # 设置cmap
    colors = ['red', 'blue', 'green', 'yellow', 'black', 'purple', 'pink', 'orange', 'gray', 'brown', 'cyan', 'magenta',
              'olive', 'lime', 'teal', 'navy']
    fig, axes = plt.subplots(1,2,figsize=(30,6))
    fig.suptitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
    C = data.iloc[:end, 1:5].sum(axis=1)
    DISC = data.iloc[:end, 5:9].sum(axis=1)
    ADISC = data.iloc[:end, 9:13].sum(axis=1)
    D = data.iloc[:end, 13:17].sum(axis=1)
    NR = data.iloc[:end, [1, 5, 9, 13]].sum(axis=1)
    SR = data.iloc[:end, [2, 6, 10, 14]].sum(axis=1)
    AR = data.iloc[:end, [3, 7, 11, 15]].sum(axis=1)
    UR = data.iloc[:end, [4, 8, 12, 16]].sum(axis=1)

    axes[0].set_title("donor")
    color_index = 0
    axes[0].plot(data.iloc[:end, 0], C, label="C", color=colors[color_index])
    color_index += 1
    axes[0].plot(data.iloc[:end, 0], DISC, label="DISC", color=colors[color_index])
    color_index += 1
    axes[0].plot(data.iloc[:end, 0], ADISC, label="ADISC", color=colors[color_index])
    color_index += 1
    axes[0].plot(data.iloc[:end, 0], D, label="D", color=colors[color_index])
    axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, 1.20), ncol=4, fancybox=True, shadow=True)

    axes[1].set_title("recipient")
    color_index = 0
    axes[1].plot(data.iloc[:end, 0], NR, label="NR", color=colors[color_index])
    color_index += 1
    axes[1].plot(data.iloc[:end, 0], SR, label="SR", color=colors[color_index])
    color_index += 1
    axes[1].plot(data.iloc[:end, 0], AR, label="AR", color=colors[color_index])
    color_index += 1
    axes[1].plot(data.iloc[:end, 0], UR, label="UR", color=colors[color_index])
    axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, 1.20), ncol=4, fancybox=True, shadow=True)

In [ ]:
for log in filtered_logs:
    # 读取数据
    data = log.get_data(NROWS)
    # 第一列为横轴，第二列为纵轴画曲线，label为列名
    # 设置cmap
    colors = ['red', 'blue', 'green', 'yellow', 'black', 'purple', 'pink', 'orange', 'gray', 'brown', 'cyan', 'magenta',
              'olive', 'lime', 'teal', 'navy']
    # 设置两个子图，每个大小为 10 * 3
    fig, axes = plt.subplots(1, 3, figsize=(30, 5))
    fig.suptitle("normId: " + str(log["normId"]))
    fig.subplots_adjust(top=0.8)
    for i in range(17, 20):
        index = i - 17
        axes[index].set_title(data.columns.values[i])
        axes[index].set_xlabel("episode")
        axes[index].set_ylabel(data.columns.values[i])
        axes[index].plot(data.iloc[:end, 0], data.iloc[:end, i], label=data.columns.values[i], color=colors[index - 1])
    # fig.legend(loc='upper center', bbox_to_anchor=(0.5, 1.30), ncol=4, fancybox=True, shadow=True)
    # plt.savefig('./norm10-c-ur.png', dpi=600)

In [ ]:
!jupyter nbconvert --to markdown visualize_json_rl_all_log.ipynb

In [ ]:
!pandoc visualize_json_rl_all_log.md -o visualize_json_rl_all_log.pdf --pdf-engine=xelatex -V mainfont="SimSun" -V geometry:margin=1in